# Spike for LangChain RAG

## Install libary
```shell
cd ../backend
poetry add langchain
poetry add langchain_community
poetry add langchain_chroma
poetry add langchain-openai
```

In [1]:
## setup langchain openai client

from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="glm-4-air", base_url="https://open.bigmodel.cn/api/paas/v4/", api_key=os.environ.get("ZHIPU_API_KEY"))

[2024-09-09 14:49:09 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:09 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:09 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:09 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:10 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:10 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:10 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:10 - httpx:146 - 

In [2]:
from langchain_openai import OpenAIEmbeddings

embedding_with_openai_embedding_client = OpenAIEmbeddings(model="embedding-3", base_url="https://open.bigmodel.cn/api/paas/v4/", api_key=os.environ.get("ZHIPU_API_KEY"))

[2024-09-09 14:49:12 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:12 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:12 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:12 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:12 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:12 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 14:49:12 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 14:49:12 - httpx:146 - 

In [3]:
## preview

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
# why use splits[0:40], because zhipu embedding model text not long as openai
vectorstore = Chroma.from_documents(documents=splits[0:40], embedding=embedding_with_openai_embedding_client)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

[2024-09-09 14:49:18 - langchain_community.utils.user_agent:11 - WARNING] USER_AGENT environment variable not set, consider setting it to identify your requests.
[2024-09-09 14:49:19 - openai._base_client:446 - DEBUG] Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x1193594e0>, 'json_data': {'input': [[4178, 44, 60720, 97548, 51354, 7361, 1956, 25, 5651, 220, 1419, 11, 220, 2366, 18, 220, 765, 220, 83086, 18242, 4212, 25, 220, 2148, 1332, 220, 765, 220, 7030, 25, 41578, 1122, 468, 833, 1432, 31233, 13307, 449, 445, 11237, 320, 17185, 4221, 1646, 8, 439, 1202, 6332, 6597, 374, 264, 7155, 7434, 13, 26778, 11311, 8838, 15204, 58871, 68353, 11, 1778, 439, 9156, 38, 2898, 11, 480, 2898, 12, 4674, 261, 323, 21266, 1929, 40, 11, 8854, 439, 34147, 10507, 13, 578, 4754, 488, 315, 445, 11237, 2289, 7953, 24038, 1664, 67383, 11236, 11, 7493, 11, 23691, 323, 7620, 26, 433, 649, 387, 47093, 439, 264, 8147, 4689,

"Task Decomposition is the process by which a large task is broken down into smaller, more manageable subtasks, as seen in the LLM's task planning stage where user requests are parsed into multiple tasks with specific attributes."